In [ ]:
# create demand file
import csv
import json
import pandas as pd
import datetime

# read csv file
with open('../data/raw/load_june_w3_2025.csv', mode='r', newline='') as f:
    reader = csv.DictReader(f)
    data = [row for row in reader]

# create demand list
demands = []
for row in data:
    # Parse the original time format and convert to ISO
    time_str = row['MTU (CET/CEST)'].split('-')[1].strip()  # Get time part after dash
    dt = pd.to_datetime(time_str, format='%d/%m/%Y %H:%M')
    iso_time = dt.strftime('%Y-%m-%d %H:%M:%S')
    
    demand = {
        'demand': float(row['Actual Total Load (MW)']),
        'time': iso_time
    }
    demands.append(demand)

# Convert to DataFrame and save as CSV
demands_df = pd.DataFrame(demands)
demands_df.to_csv('../data/clean/demand.csv', index=False)

In [3]:
## Create Global Variables file
import csv
import json

# Load fuel prices from a CSV file
with open('../data/raw/fuel_prices.csv', mode='r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    fuel_prices = [row for row in reader]

# save the last row as the latest fuel prices
latest_fuel_prices = fuel_prices[-1]

# drop time
latest_fuel_prices.pop('time', None)

# add renewable technologies
latest_fuel_prices['Wind Onshore'] = 0.0
latest_fuel_prices['Wind Offshore'] = 0.0
latest_fuel_prices['Solar'] = 0.0
latest_fuel_prices['Biomass'] = 0.0

print("Latest fuel prices with renewables:", latest_fuel_prices.keys())

for tech in latest_fuel_prices.keys():
    if tech == 'EUA':
        continue  # Skip EUA as it is handled separately
    elif tech == 'Fossil Gas':
        latest_fuel_prices[tech] = {
            'fuel_cost': float(latest_fuel_prices[tech]),
            'co2_intensity': 0.20,
            'eff_high': 0.40,
            'eff_low': 0.25,
            'other_costs': 0.0, 
            'carbon_cost': latest_fuel_prices['EUA'],
            'emoji': '🔥'
        }
    elif tech == 'Fossil Hard coal':
        latest_fuel_prices[tech] = {
            'fuel_cost': float(latest_fuel_prices[tech]),
            'co2_intensity': 0.30,
            'eff_high': 0.46,
            'eff_low': 0.35,
            'other_costs': 0.0,
            'carbon_cost': latest_fuel_prices['EUA'],
            'emoji': '🪨'
        }
    elif tech == 'Nuclear':
        latest_fuel_prices[tech] = {
            'fuel_cost': float(latest_fuel_prices[tech]),
            'co2_intensity': 0.03,
            'eff_high': 0.42,
            'eff_low': 0.32,
            'other_costs': 0.0,
            'carbon_cost': latest_fuel_prices['EUA'],
            'emoji': '⚛️'
        }
    elif tech == 'Biomass':
        latest_fuel_prices[tech] = {
            'fuel_cost': float(latest_fuel_prices[tech]),
            'co2_intensity': 0.0,
            'eff_high': 1.0,
            'eff_low': 1.0,
            'other_costs': 0.0,
            'carbon_cost': latest_fuel_prices['EUA'],
            'emoji': '🌱'
        }
    else:
        latest_fuel_prices[tech] = {
            'fuel_cost': 0.0,
            'co2_intensity': 0.0,
            'eff_high': 1.0,
            'eff_low': 1.0,
            'other_costs': 0.0,
            'carbon_cost': latest_fuel_prices['EUA'],
            'emoji': '❓' # emoji to represent wind turbines is not available
        }
latest_fuel_prices.pop('EUA', None)

with open('../data/clean/power_tech_parameters.json', 'w', encoding='utf-8') as f:
    json.dump(latest_fuel_prices, f, indent=4)

Latest fuel prices with renewables: dict_keys(['Fossil Gas', 'Nuclear', 'Fossil Hard coal', 'EUA', 'Wind Onshore', 'Wind Offshore', 'Solar', 'Biomass'])


In [4]:
# Create energy source files
import csv
import json

# read csv file

with open('../data/raw/Installed Capacity Per Production Unit_2025.csv', mode='r', newline='') as f:
    reader = csv.DictReader(f)
    data = [row for row in reader]

with open('../data/clean/power_tech_parameters.json', mode='r', newline='') as f:
    tech_params = json.load(f)

key_mapping = {
    'Production Type': 'tech_name',
    'Name': 'name',
    'Current Installed Capacity [MW]': 'capacity'
}

# apply key mapping
mapped_data = [
    {key_mapping[k]: v for k, v in row.items() if k in key_mapping} 
    for row in data
]

# insert technology parameters based on tech_name
for row in mapped_data:
    tech_name = row['tech_name']
    if tech_name in tech_params:
        row.update(tech_params[tech_name])
    else:
        print(f"Warning: No parameters found for technology '{tech_name}'")

# Add index to each source
for i, source in enumerate(mapped_data):
    source['idx'] = i


# write to json file
with open('../data/clean/sources.json',
        mode='w', encoding='utf-8') as f:
        json.dump(mapped_data, f, indent=4)